<a href="https://colab.research.google.com/github/jrmdsgr/linear-regression-health-costs-calculator/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], drop_first=True)

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

numeric_features = ['age', 'bmi', 'children']
train_stats = train_dataset[numeric_features].describe().transpose()

def normalize(x):
    return (x - train_stats['mean']) / train_stats['std']

norm_train_data = train_dataset.copy()
norm_test_data = test_dataset.copy()
norm_train_data[numeric_features] = normalize(norm_train_data[numeric_features])
norm_test_data[numeric_features] = normalize(norm_test_data[numeric_features])

model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(norm_train_data.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    norm_train_data, train_labels,
    validation_split=0.2,
    epochs=500,
    batch_size=32,
    verbose=0,
    callbacks=[early_stop]
)

train_dataset = norm_train_data
test_dataset = norm_test_data

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)